# Processing Tabular Data with Spark SQL

The dataset to be used is from eBay online auctions. The eBay online auction dataset contains the following fields:

    auctionid - unique identifier of an auction
    bid - the proxy bid placed by a bidder
    bidtime - the time (in days) that the bid was placed, from the start of the auction
    bidder - eBay username of the bidder
    bidderrate - eBay feedback rating of the bidder
    openbid - the opening bid set by the seller
    price - the closing price that the item sold for (equivalent to the second highest bid + an increment)


Using Spark DataFrames, we will explore the eBay data with questions like:

    How many auctions were held?
    How many bids were made per item?
    What's the minimum, maximum, and average number of bids per item?
    Show the bids with price > 100


Start by loading the data from the ebay.csv file into a Resilient Distributed Dataset (RDD). RDDs have transformations and actions; the first() action returns the first element in the RDD:

In [98]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Row

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [99]:
# load the data into a new RDD
ebay_text = sc.textFile("./raw/ebay.csv")

# Return the first element in this RDD
ebay_text.first()

u'8213034705,95,2.927373,jake7870,0,95,117.5,xbox,3'

Now we need to define the Auction schema corresponding to the ebay.csv file. Then a map() transformation is applied to each element of ebayText to create the ebay RDD of Auction objects.

In [100]:
# Define the schema using StructType
ebay_schema = StructType([StructField("auctionid", StringType(), False),
                             StructField("bid", FloatType(), False),
                             StructField("bidtime", FloatType()),
                             StructField("bidder", StringType()),
                             StructField("bidderrate", IntegerType()),
                             StructField("openbid", FloatType()),
                             StructField("price", FloatType()),
                             StructField("item", StringType()),
                             StructField("daystolive", IntegerType())
                            ])

# create an RDD of Auction objects
# convert each line of csv file to a Row
ebay_rdd = ebay_text.map(lambda line: line.split(",")) \
                    .map(lambda p: Row(auctionid=p[0], bid=float(p[1]),
                                       bidtime=float(p[2]), bidder=p[3],
                                       bidderrate=int(p[4]), openbid=float(p[5]),
                                       price=float(p[6]), item=p[7], daystolive=int(p[8])))
# Create a DataFrame from RDD and schema
ebay_df = spark.createDataFrame(ebay_rdd, ebay_schema)

# Another way to create DataFrame 
#ebay_df = ebay_rdd.toDF(['auctionid', 'bid', 'bidtime', 'bidder', 'bidderrate', 'openbid', 'price', 'item', 'daystolive'])

In [101]:
#Show top 20 rows of ebay_df DataFrame
ebay_df.show()

+----------+-----+--------+--------------+----------+-------+-----+----+----------+
| auctionid|  bid| bidtime|        bidder|bidderrate|openbid|price|item|daystolive|
+----------+-----+--------+--------------+----------+-------+-----+----+----------+
|8213034705| 95.0|2.927373|      jake7870|         0|   95.0|117.5|xbox|         3|
|8213034705|115.0|2.943484| davidbresler2|         1|   95.0|117.5|xbox|         3|
|8213034705|100.0|2.951285|gladimacowgirl|        58|   95.0|117.5|xbox|         3|
|8213034705|117.5|2.998947|       daysrus|        10|   95.0|117.5|xbox|         3|
|8213060420|  2.0|0.065266|    donnie4814|         5|    1.0|120.0|xbox|         3|
|8213060420|15.25|0.123218|   myreeceyboy|        52|    1.0|120.0|xbox|         3|
|8213060420|  3.0|0.186539|  parakeet2004|         5|    1.0|120.0|xbox|         3|
|8213060420| 10.0| 0.18669|  parakeet2004|         5|    1.0|120.0|xbox|         3|
|8213060420|24.99|0.187049|  parakeet2004|         5|    1.0|120.0|xbox|    

In [102]:
# Display the schema of this DataFrame
ebay_df.printSchema()

root
 |-- auctionid: string (nullable = false)
 |-- bid: float (nullable = false)
 |-- bidtime: float (nullable = true)
 |-- bidder: string (nullable = true)
 |-- bidderrate: integer (nullable = true)
 |-- openbid: float (nullable = true)
 |-- price: float (nullable = true)
 |-- item: string (nullable = true)
 |-- daystolive: integer (nullable = true)



In [103]:
# How many auctions were held?
ebay_df.select(ebay_df.auctionid).distinct().count()

#Another way to compute the number of auctions
#from pyspark.sql.functions import *
#ebay_df.agg(countDistinct(ebay_df.auctionid)).collect()

627

In [104]:
# How many bids per item?
ebay_df.groupBy('auctionid', 'item').count().show()

+----------+-------+-----+
| auctionid|   item|count|
+----------+-------+-----+
|1645883276|cartier|   12|
|1641242797|cartier|    5|
|1642005995|cartier|   18|
|3020060476|   palm|   22|
|1644681469|cartier|   14|
|8212145833|   xbox|   26|
|1638844464|cartier|   16|
|3020573049|   palm|    2|
|3015632013|   palm|   28|
|3024659380|   palm|   26|
|3025373736|   palm|   15|
|8213387659|   xbox|   25|
|1643201832|cartier|    4|
|3014834745|   palm|   21|
|3020257463|   palm|   16|
|3020274575|   palm|   11|
|3019821569|   palm|   27|
|3024707992|   palm|   20|
|8212277835|   xbox|    3|
|3019326870|   palm|   33|
+----------+-------+-----+
only showing top 20 rows



In [105]:
# Get the auctions with closing price > 100
highprice = ebay_df.filter("price > 100")

# display dataframe in a tabular format
highprice.show()

+----------+-----+--------+--------------+----------+-------+-----+----+----------+
| auctionid|  bid| bidtime|        bidder|bidderrate|openbid|price|item|daystolive|
+----------+-----+--------+--------------+----------+-------+-----+----+----------+
|8213034705| 95.0|2.927373|      jake7870|         0|   95.0|117.5|xbox|         3|
|8213034705|115.0|2.943484| davidbresler2|         1|   95.0|117.5|xbox|         3|
|8213034705|100.0|2.951285|gladimacowgirl|        58|   95.0|117.5|xbox|         3|
|8213034705|117.5|2.998947|       daysrus|        10|   95.0|117.5|xbox|         3|
|8213060420|  2.0|0.065266|    donnie4814|         5|    1.0|120.0|xbox|         3|
|8213060420|15.25|0.123218|   myreeceyboy|        52|    1.0|120.0|xbox|         3|
|8213060420|  3.0|0.186539|  parakeet2004|         5|    1.0|120.0|xbox|         3|
|8213060420| 10.0| 0.18669|  parakeet2004|         5|    1.0|120.0|xbox|         3|
|8213060420|24.99|0.187049|  parakeet2004|         5|    1.0|120.0|xbox|    

In [106]:
# Register the DataFrame as a temp table using a given name
ebay_df.registerTempTable("ebay_table")

# How many bids per auction?
results = spark.sql("SELECT auctionid, item, count(bid) FROM ebay_table GROUP BY auctionid, item")

results.show()

results = spark.sql("SELECT auctionid, MAX(price) FROM ebay_table GROUP BY item, auctionid")
results.show()

+----------+-------+----------+
| auctionid|   item|count(bid)|
+----------+-------+----------+
|1645883276|cartier|        12|
|1641242797|cartier|         5|
|1642005995|cartier|        18|
|3020060476|   palm|        22|
|1644681469|cartier|        14|
|8212145833|   xbox|        26|
|1638844464|cartier|        16|
|3020573049|   palm|         2|
|3015632013|   palm|        28|
|3024659380|   palm|        26|
|3025373736|   palm|        15|
|8213387659|   xbox|        25|
|1643201832|cartier|         4|
|3014834745|   palm|        21|
|3020257463|   palm|        16|
|3020274575|   palm|        11|
|3019821569|   palm|        27|
|3024707992|   palm|        20|
|8212277835|   xbox|         3|
|3019326870|   palm|        33|
+----------+-------+----------+
only showing top 20 rows

+----------+----------+
| auctionid|max(price)|
+----------+----------+
|8212264580|    127.51|
|1649848613|    1100.0|
|1647870862|    1250.0|
|1639425502|     224.5|
|8211480551|     311.6|
|8214749544|  